# Visualization pySCENIC output

The present study is based on the 10X scRNA-seq dataset published by the Allen Institute for Brain Science and publicly available at: https://portal.brain-map.org/atlases-and-data/RNA-seq/mouse-whole-cortex-and-hippocampus-10x. The data was then clustered, and cluster names were assigned based on the Allen Institute proposal for cell type nomenclature (https://portal.brain-map.org/explore/classes/nomenclature). The topology of the taxonomy allowed to define the sex of the mouse from which the cells were isolated, the regions of interest, cell classes (glutamatergic, GABAergic or Non-Neuronal) and subclasses. This information was stored in the metadata table. The metadata was used to subset cells of the hippocampus region from the gene expression matrix. We selected for 13 subclasses of hippocampal cells. The hippocampus gene count matrix was pre-processed in R v3.6.1 according to the Seurat v3.1.5 standard pre-processing workflow for quality control, normalization, and analysis of scRNA-seq data (cf. 10XHip2021_Pre.Processing). It was transform in a loom object and used as input for the pySCENIC workflow (cf. 10XHip2021_4.pySCENIC-workflow).

# Description

Here we describe how we obtained the figures for gene regulatory network activity, using pySCENIC output.

# Data availability

cf. README to download pySCENIC output '10XHip2021_GRN.matrix.csv'

# Pre-processing

### 1. Load data and required libraries

In [ ]:
# Load libraries
library(dplyr)
library(Seurat)
library(tidyverse)
library(ggplot2)

In [ ]:
# Load gene regulatory network (GRN) matrix
GRN <- read.delim2("/path/10XHip2021_GRN.matrix.csv", sep = ',')
row.names(GRN) <- GRN$Cell
GRN <- GRN[-1]

# Load original metadata from the portal brain map website
metadata <- read.delim2("/path/file_name.csv", sep = ',')
row.names(metadata) <- metadata$X
metadata <- metadata[-1]

### 2. Replace column names by gene names (meaning: remove the (+) sign)

In [4]:
names(GRN) <- str_replace(names(GRN),"\\.\\.\\.","\\(\\+\\)")

### 3. Create Seurat object and AddMetaData

In [ ]:
# Create Seurat object
hip_reg <- CreateSeuratObject(counts = t(GRN)) # Total of 379 GRNs

In [ ]:
# AddMetaData
# Class
classLabels <- metadata$class
names(classLabels) <- row.names(metadata)
hip_reg <- AddMetaData(hip_reg, classLabels, col.name='class')
# Subclass
subclassLabels <- metadata$subclass
names(subclassLabels) <- row.names(metadata)
hip_reg <- AddMetaData(hip_reg, subclassLabels, col.name='subclass')
# Cluster
clusterLabels <- metadata$cluster
names(clusterLabels) <- row.names(metadata)
hip_reg <- AddMetaData(hip_reg, clusterLabels, col.name='cluster')

### 4. Normalization

In [ ]:
# Normalization of the data
hip_reg <- NormalizeData(hip_reg, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
# Identification of the high variable features (genes)
hip_reg <- FindVariableFeatures(hip_reg, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(hip_reg), 10)

# Plot variable features with and without labels
plot1 <- VariableFeaturePlot(hip_reg)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE, xnudge = 0, ynudge = 0)

### 5. Scaling and linear dimensional reduction

In [ ]:
# Scaling the data
all.genes <- rownames(hip_reg)
hip_reg <- ScaleData(hip_reg, features = all.genes)

In [ ]:
# Perform linear dimensional reduction
hip_reg <- RunPCA(hip_reg, features = VariableFeatures(object = hip_reg))
#PCA
PCA <- DimPlot(hip_reg, reduction = "pca", cols = 'black')

### 6. Clustering and t-SNE

In [ ]:
# Cluster the cells
hip_reg <- FindNeighbors(hip_reg, dims = 1:50)
hip_reg <- FindClusters(hip_reg, resolution = 0.5)

In [ ]:
# Annotate cells by subclasses
Idents(object = hip_reg) <- "subclass"

In [ ]:
# t-SNE reduction
hip_reg <- RunTSNE(hip_reg, reduction = "pca", dims = 1:30, perplexity = 30, max_iter = 1000)

In [ ]:
# Save final object as RDS file
saveRDS(hip_reg, file = "/path/file_name.rds")

# Expression gene regulatory network (GRN) activity

### 1. t-SNE map

In [ ]:
DimPlot(hip_reg, reduction = "tsne", label = TRUE, pt.size=0.5, label.size = 6, 
                          cols=c('CA1-ProS'='skyblue', 'CA2'='lightseagreen', 'CA3'='steelblue', 
                                 'DG'='slategray2','Lamp5'='violetred4','Pvalb'='mediumvioletred',
                                 'Sncg'='palevioletred1','Sst'='pink1','Vip'='palevioletred3', 
                                 'Endo'='forestgreen', 'Micro-PVM'='yellowgreen', 'Oligo'='orange2', 
                                 'Astro'='sienna3'))  + xlim(-40,40) + ylim(-40,40) + 
                          theme(axis.title.x=element_text(size=40), 
                          axis.title.y=element_text(size=40), axis.text.x = element_text(size = 40), 
                          axis.text.y = element_text(size = 40)) + NoLegend()

### 2. Differential activity analysis: which GRNs are more active in which cell types?

In [ ]:
wilcox.enrichment <- FindAllMarkers(object = hip_reg, test.use = 'wilcox', 
                                    logfc.threshold = 0.25, min.pct = 0.1, only.pos = TRUE)

### 3. Display a network activity as t-SNE or violin plot

In [ ]:
network = "GRN_of_interest(+)"
# tSNE (relative expression - scale from 1 to 3 = low/medium/high)

tSNE.hip = FeaturePlot(hip_reg, features = network, cols = c("lightgrey","mediumpurple1","blue"), 
            coord.fixed = TRUE) + xlim(-40,40) + ylim(-40,40) + theme(axis.title.x=element_text(size=40),
            title = element_text(size=48), axis.title.y=element_text(size=40), 
            axis.text.x = element_text(size = 40), axis.text.y = element_text(size = 40), 
            legend.title = element_text(size=40),legend.text = element_text(size=20))

# Violin plot
vlnplot = VlnPlot(hip_reg, features = network, cols = c('CA1-ProS'='skyblue', 'CA2'='lightseagreen', 'CA3'='steelblue', 
          'DG'='slategray2','Lamp5'='violetred4','Pvalb'='mediumvioletred','Sncg'='palevioletred1','Sst'='pink1',
          'Vip'='palevioletred3','Endo'='forestgreen', 'Micro-PVM'='yellowgreen', 'Oligo'='orange2', 
          'Astro'='sienna3'), pt.size = 0.1) + theme(axis.title.x=element_blank(),title = element_text(size=48), 
           axis.title.y=element_text(size=40),axis.text.x = element_text(size = 40), 
           axis.text.y = element_text(size = 40), legend.title = element_text(size=40),
           legend.text = element_text(size=20)) + NoLegend()

### 4. Heatmap of all GRNs

In [ ]:
# Networks of interest 
networks <- c('2010315B03Rik(+)','4921501E09Rik(+)','6720489N17Rik(+)','AI987944(+)','Arid3a(+)','Aspscr1(+)',
              'Atf3(+)','Atf4(+)','Atf6b(+)','B230307C23Rik(+)','Bach1(+)','Bach2(+)','Barhl2(+)','Batf(+)',
              'Bcl3(+)','Bclaf1(+)','Bhlhe22(+)','Bhlhe23(+)','Bhlhe40(+)','Bhlhe41(+)','Bmyc(+)','Brf1(+)',
              'Brf2(+)','Cbfb(+)','Cebpa(+)','Cebpb(+)','Cebpd(+)','Cebpz(+)','Chd1(+)','Chd2(+)','Clock(+)',
              'Cpsf4(+)','Creb3(+)','Creb3l1(+)','Creb3l2(+)','Crem(+)','Ctcf(+)','Cux1(+)','Dbp(+)','Dbx2(+)',
              'Ddit3(+)','Deaf1(+)','Dlx1(+)','Dlx2(+)','Dlx5(+)','Dlx6(+)','Dmrta2(+)','E2f1(+)','E2f3(+)',
              'E2f4(+)','E2f6(+)','E2f7(+)','E4f1(+)','Ebf1(+)','Egr1(+)','Egr2(+)','Egr3(+)','Egr4(+)','Elf1(+)',
              'Elf4(+)','Elk1(+)','Elk3(+)','Elk4(+)','Emx2(+)','Eomes(+)','Ep300(+)','Erf(+)','Erg(+)','Esr2(+)',
              'Esrra(+)','Esrrg(+)','Ets1(+)','Etv6(+)','Ewsr1(+)','Ezh2(+)','Fank1(+)','Fli1(+)','Fos(+)',
              'Fosb(+)','Fosl1(+)','Fosl2(+)','Foxc1(+)','Foxf2(+)','Foxg1(+)','Foxh1(+)','Foxj3(+)','Foxk1(+)',
              'Foxn2(+)','Foxn3(+)','Foxo1(+)','Foxo3(+)','Foxo4(+)','Foxp1(+)','Foxp2(+)','Foxq1(+)','Fubp1(+)',
              'Gabpa(+)','Gabpb1(+)','Gadd45a(+)','Gata2(+)','Gata6(+)','Glis2(+)','Gm14326(+)','Gm14403(+)',
              'Gm43517(+)','Gmeb1(+)','Gmeb2(+)','Grhl1(+)','Gsc2(+)','Gsx1(+)','Gsx2(+)','Gtf2f1(+)','Hbp1(+)',
              'Hcfc1(+)','Hdac2(+)','Hdac3(+)','Hdac6(+)','Hes1(+)','Hey1(+)','Hic1(+)','Hlf(+)','Hmga1(+)',
              'Hnf1a(+)','Hsf3(+)','Hspa5(+)','Ikzf1(+)','Irf1(+)','Irf2(+)','Irf3(+)','Irf4(+)','Irf5(+)',
              'Irf7(+)','Irf9(+)','Irx2(+)','Irx5(+)','Jdp2(+)','Jun(+)','Junb(+)','Jund(+)','Kat2a(+)','Kdm5a(+)',
              'Kdm5b(+)','Klf10(+)','Klf11(+)','Klf13(+)','Klf14(+)','Klf16(+)','Klf2(+)','Klf3(+)','Klf4(+)',
              'Klf6(+)','Klf8(+)','Klf9(+)','Klrg1(+)','Lef1(+)','Lhx1(+)','Lhx5(+)','Lhx6(+)','Lhx9(+)','Maf(+)',
              'Mafb(+)','Maff(+)','Mafg(+)','Max(+)','Maz(+)','Mbd4(+)','Mecom(+)','Mef2a(+)','Mef2b(+)','Mef2c(+)',
              'Mef2d(+)','Meis1(+)','Meis3(+)','Mkx(+)','Mlx(+)','Mlxip(+)','Mnt(+)','Msc(+)','Mxi1(+)','Myc(+)',
              'Mycn(+)','Mylk(+)','Nelfe(+)','Neurod2(+)','Nfat5(+)','Nfatc1(+)','Nfatc4(+)','Nfe2l1(+)',
              'Nfe2l2(+)','Nfe2l3(+)','Nfia(+)','Nfic(+)','Nfil3(+)','Nfkb1(+)','Nfya(+)','Nfyb(+)','Nhlh1(+)',
              'Nkx2.1(+)','Nkx2.2(+)','Nkx6.2(+)','Nkx6.3(+)','Npdc1(+)','Nr1d1(+)','Nr1h2(+)','Nr1h3(+)',
              'Nr1h4(+)','Nr2c2(+)','Nr2f6(+)','Nrf1(+)','Nrl(+)','Odc1(+)','Olig2(+)','Onecut2(+)','Osr1(+)',
              'Otx1(+)','Otx2(+)','Pax2(+)','Pax5(+)','Pax6(+)','Pbx1(+)','Pdlim5(+)','Phf8(+)','Phox2a(+)',
              'Pir(+)','Pml(+)','Pold2(+)','Polr3a(+)','Polr3g(+)','Pou2f1(+)','Pou2f3(+)','Pou3f1(+)','Pou3f2(+)',
              'Pou3f3(+)','Pou4f1(+)','Pou6f1(+)','Ppara(+)','Ppard(+)','Ppargc1a(+)','Pqbp1(+)','Prdm1(+)',
              'Prdm14(+)','Prdm16(+)','Prdm4(+)','Prdm9(+)','Prox1(+)','Psma6(+)','Psmd12(+)','Rad21(+)','Rara(+)',
              'Rarg(+)','Rbbp5(+)','Rbm7(+)','Rcor1(+)','Rel(+)','Rela(+)','Rest(+)','Rfx7(+)','Rfxap(+)','Rora(+)',
              'Rreb1(+)','Runx1(+)','Ruvbl1(+)','Rxra(+)','Sall2(+)','Sall4(+)','Sap30(+)','Setdb1(+)','Sin3a(+)',
              'Sirt6(+)','Six4(+)','Smad1(+)','Smad5(+)','Smad6(+)','Smad9(+)','Smarca4(+)','Smarcc1(+)',
              'Smarcc2(+)','Smc3(+)','Sox1(+)','Sox10(+)','Sox11(+)','Sox12(+)','Sox13(+)','Sox17(+)','Sox2(+)',
              'Sox21(+)','Sox4(+)','Sox8(+)','Sox9(+)','Sp1(+)','Sp2(+)','Sp3(+)','Sp5(+)','Spi1(+)','Srebf1(+)',
              'Srebf2(+)','Srf(+)','Stat1(+)','Stat2(+)','Stat3(+)','Stat5a(+)','Stat5b(+)','Stat6(+)','Supt20(+)',
              'Taf1(+)','Tal1(+)','Tbl1xr1(+)','Tbp(+)','Tbr1(+)','Tbx21(+)','Tbx3(+)','Tbx4(+)','Tcf12(+)',
              'Tcf3(+)','Tcf7l1(+)','Tcf7l2(+)','Tead1(+)','Tead2(+)','Tead4(+)','Tef(+)','Tfdp1(+)','Tfe3(+)',
              'Tfeb(+)','Thap1(+)','Thap11(+)','Traf4(+)','Trip10(+)','Ubp1(+)','Usf1(+)','Usf2(+)','Vax1(+)',
              'Vsx2(+)','Xbp1(+)','Xrcc4(+)','Yy1(+)','Zbtb17(+)','Zbtb43(+)','Zbtb46(+)','Zbtb7a(+)','Zc3h7a(+)',
              'Zeb1(+)','Zfp110(+)','Zfp128(+)','Zfp143(+)','Zfp239(+)','Zfp263(+)','Zfp286(+)','Zfp30(+)',
              'Zfp335(+)','Zfp354a(+)','Zfp369(+)','Zfp46(+)','Zfp523(+)','Zfp558(+)','Zfp560(+)','Zfp595(+)',
              'Zfp597(+)','Zfp607b(+)','Zfp62(+)','Zfp64(+)','Zfp646(+)','Zfp729b(+)','Zfp740(+)','Zfp770(+)',
              'Zfp800(+)','Zfp825(+)','Zfp90(+)','Zfp932(+)','Zfp963(+)','Zfp97(+)','Zic1(+)','Zic2(+)','Zic3(+)',
              'Zic4(+)','Zkscan1(+)','Zmat4(+)','Zmiz1(+)','Zscan29(+)','Zxdb(+)')

length(networks) # 379

In [ ]:
# Create dataset from GRN activity 
data <- DotPlot(hip_reg, features = networks)$data

In [ ]:
# One dataset per cell type
Lamp5 <- data$avg.exp[data$id == 'Lamp5']
Sncg <- data$avg.exp[data$id == 'Sncg']
Sst <- data$avg.exp[data$id == 'Sst']
Vip <- data$avg.exp[data$id == 'Vip']
Pvalb <- data$avg.exp[data$id == 'Pvalb'] 
DG <- data$avg.exp[data$id == 'DG']
CA1ProS <- data$avg.exp[data$id == 'CA1-ProS']
CA2 <- data$avg.exp[data$id == 'CA2']
CA3 <- data$avg.exp[data$id == 'CA3']
Oligo <- data$avg.exp[data$id == 'Oligo']
Astro <- data$avg.exp[data$id == 'Astro']
Endo <- data$avg.exp[data$id == 'Endo']
MicroPVM <- data$avg.exp[data$id == 'Micro-PVM']

# Names GRNs
GRN_names <- data$features.plot[data$id == 'Lamp5'] # any cell population will have all GRN names

# Cells of interest
cells <- c("Lamp5","Sncg","Sst","Vip","Pvalb","DG","CA1-ProS","CA2","CA3","Oligo","Astro","Endo","Micro-PVM")

In [ ]:
# Creation of the matrix
mat <- cbind(Lamp5,Sncg,Sst,Vip,Pvalb,DG,CA1ProS,CA2,CA3,Oligo,Astro,Endo,MicroPVM)
rownames(mat) <- GRN_names
colnames(mat) <- cells

In [ ]:
# Colors heatmaps - Coupling score legend
cols = colorRamp2(c(0,100,200,300,400), c('#eff3ff','#bdd7e7','#6baed6','#3182bd','#08519c'))

# Heatmap all GRNs in hippocampal cell types
heatmap <- Heatmap(mat, cluster_rows = FALSE, row_names_side = "right", rect_gp = gpar(col = "white", lwd = 1),
                      cluster_columns = FALSE, name = "Network activity", 
                      row_names_gp = gpar(fontsize = 12), column_names_gp = gpar(fontsize = 30), 
                      col = cols, na_col="white")